<a href="https://colab.research.google.com/github/vmartinezalvarez/Deep-Learning-Topological-Invariants/blob/master/Machine_Learning_Topological_Invariants_with_Neural_Networks_and_data_set_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Generation using a general Hamiltonian. 

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

The Su-Schrieffer-Heeger (SSH) Hamiltonian is
$$
H_{SSH}(k)=(t+t'\cos k)\sigma_{x}+(t'\sin k)\sigma_{y}.
$$
This model hosts two topologically distinct gapped phases with winding
number $w=0$ for $t>t'$ and $w=1$ for $t<t'$ , respectively. To
examine whether the neural networks have the ability to learn the
winding number in its most general form, we generate training data
with the most general one-dimensional Hamiltonians with chiral symmetry
$$
H(k)=h_{x}(k)\sigma_{x}+h_{y}(k)\sigma_{y},
$$
where $h_{i}(k)$, $i=x,y$ are periodic functions in $k$ expanded
by the Fourier series
$$
h_{i}(k)=\sum_{n=0}^{4}[a_{i,n}\cos(nk)+b_{i,n}\sin(nk)]
$$
c is a cutoff that determines the highest possible winding number
of the Hamiltonian, and is set to c = 4 in the following. $a_{i,n}$
and $b_{i,n}$ are randomly sampled from a uniform distribution within $[-1, 1]$


In [0]:
numk = 33
kk = np.linspace(0, 2*np.pi, num = numk )
kk[-1] = kk[0]

n_train = 100000
n_test  = 20000 

n_casos= int(n_train+n_test)

data_G = np.zeros( (numk , n_casos*2) )


count = 0

for i in range(n_casos):
    
    # "Hamiltonian parameters"
    max_c = 2

    axx=(np.random.random( max_c+1 ) *2) -1
    bxx=(np.random.random( max_c+1 ) *2) -1

    ayy=(np.random.random( max_c+1 ) *2) -1
    byy=(np.random.random( max_c+1 ) *2) -1
    ##
    
    hx_list= []
    hy_list= []

    for k in (kk) :

        hnx  = np.sum([ax*np.cos(i*k) + bx*np.sin(i*k) for i, (ax,bx) in enumerate (zip(axx,bxx))] )
        hny  = np.sum([ay*np.cos(i*k) + by*np.sin(i*k) for i, (ay,by) in enumerate (zip(ayy,byy))] )

        hx= hnx / np.sqrt(hnx*hnx+hny*hny) 
        hy= hny / np.sqrt(hnx*hnx+hny*hny) 

        hx_list.append(hx)
        hy_list.append(hy)

    data_G[:, count*2 ]  = hx_list
    data_G[:, (count*2)+1] = hy_list

    count= count+1
    
train =  data_G[:, : int(n_train*2) ]
test  =  data_G[:, int(n_train*2) : ]
np.savetxt("data_G_Train.txt" , train  )
np.savetxt("data_G_Test.txt" , test )

In [0]:
def isLeft(P0, P1, P2):
    """
    isLeft(): tests if a point is Left|On|Right of an infinite line.
    Input :  three points P0, P1, and P2
    Return: >0 for P2 left of the line through P0 and P1
            =0 for P2  on the line
            <0 for P2  right of the line
    """
    return ( (P1[0] - P0[0]) * (P2[1] - P0[1]) - (P2[0] -  P0[0]) * (P1[1] - P0[1]) )


def wn_PnPoly( P, V ):
    """
    wn_PnPoly(): winding number for a point in a polygon
    Input:   P = a point, V = vertex points of a polygon V[n+1] with V[n]=V[0]
    Return:  wn = the winding number (=0 only when P is outside)
    """   
    wn = 0;    # the  winding number counter

    # loop through all edges of the polygon
    for i in range(len(V)-1):                         # edge from V[i] to  V[i+1]  for(int i=0; i<n; i++)
        if ( V[i][1] <= P[1]):                        # start y <= P.y
            if (V[i+1][1]  > P[1]):                   # an upward crossing
                if (isLeft( V[i], V[i+1], P) > 0):    # P left of  edge
                    wn += 1;                          # have  a valid up intersect
        
        else:                                         # start y > P.y (no test needed)
            if (V[i+1][1]  <= P[1]):                  # a downward crossing
                if (isLeft( V[i], V[i+1], P) < 0):    # P right of  edge
                    wn -= 1;                          # have  a valid down intersect
        
    
    return wn

In [0]:
"COMPUTE WINDING NUMBER"

def wn(data, fname, n_train) : 
    
    list_wn= []
    
    n_casos = int( np.shape(data)[1]/ 2)
    n_k = int(len (data) )
    
    print (n_casos, n_k )

    for i in range(n_casos):
        
        hx = data[:,i*2]
        hy = data[:,(i*2)+1]
        V = np.array([hx,hy]).T
        P = (0,0)
        wn = wn_PnPoly( P, V ) 
        list_wn.append( wn)
        

    np.savetxt(fname, list_wn[:n_train], fmt='%10.1f')
    np.savetxt(fname+"_test", list_wn[n_train:], fmt='%10.1f')

In [0]:
wn(data_G, "list_wn_G.txt", n_train)

120000 33


In [0]:
wn = np.loadtxt("list_wn_G.txt")

In [0]:
index0 = np.equal(0,wn)
index1 = np.equal(1,wn)
index2 = np.equal(-1,wn)
index3 = np.equal(2,wn)
index4 = np.equal(-2,wn)


In [0]:
print ("0= " + str( (np.sum(index0)/100000)*100 ))
print ("1= " + str( (np.sum(index1)/100000)*100 ))
print ("-1= " + str( (np.sum(index2)/100000)*100 ))
print ("2= " + str( (np.sum(index3)/100000)*100 ))
print ("-2= " + str( (np.sum(index4)/100000)*100 ))

0= 52.803
1= 22.078999999999997
-1= 22.338
2= 1.349
-2= 1.431


# Convolutional Neural Network in Keras for Winding Number Calculations

#### Load dependencies

In [0]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout 
from keras.layers import Flatten, MaxPooling2D, Conv2D
from keras.layers.normalization import BatchNormalization 
from keras import regularizers 
from keras.optimizers import SGD

Using TensorFlow backend.


#### Load data

In [0]:
### parameters
numk = 33
n_train = 100000
n_test  =  20000
n_casos = 120000


### Training data ###
X_train = np.zeros( (n_train, numk, 2 ) ) 

data_raw = np.loadtxt("data_G_Train.txt")       

for i in range(n_train) :
    count = int(i*2)
    X_train[i]= np.column_stack( ( data_raw[:,count] , data_raw[:,count+1] ) )
    
y_train= np.loadtxt("list_wn_G.txt")


### Test data


X_test = np.zeros( (n_test, numk, 2 ) ) 

data_raww = np.loadtxt("data_G_Test.txt")       

for i in range(n_test) :
    count = int(i*2)
    X_test[i]= np.column_stack( ( data_raww[:,count] , data_raww[:,count+1] ) )
    

y_test= np.loadtxt("list_wn_G.txt_test")


print (X_test.shape)
print (y_test)

(20000, 33, 2)
[-1. -1. -1. ...  0.  0.  0.]


In [0]:
X_train.shape     # en nuestro caso sería (10^5, 33, 2)

(100000, 33, 2)

In [0]:
y_train.shape     # en nuestro caso sería (10^5,)

(100000,)

#### Preprocess data

In [0]:
#X_train = X_train.reshape(60000, 784).astype('float32')
#X_test = X_test.reshape(10000, 784).astype('float32')

X_train = X_train.reshape(n_train, numk, 2, 1).astype('float32')
#X_test = X_test.reshape(10000, 28, 28, 1).astype('float32')


X_test = X_test.reshape(n_test, numk, 2, 1).astype('float32')
print(X_train.shape)
print(X_test.shape)


(100000, 33, 2, 1)
(20000, 33, 2, 1)


In [0]:
model = Sequential()
model.add(Conv2D(40, kernel_size=(2, 2), activation='relu', input_shape=(33, 2, 1)))
#model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(1, kernel_size=(1, 1), activation='relu'))
model.add(Flatten())
model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='linear'))


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 1, 40)         200       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 1, 1)          41        
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
Total params: 310
Trainable params: 310
Non-trainable params: 0
_________________________________________________________________


#### Configure model

In [0]:
#model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.1), metrics=['accuracy'])
model.compile(loss='mean_squared_error',  optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Train!

In [0]:
model.fit(X_train, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(X_test, y_test))

Train on 100000 samples, validate on 20000 samples
Epoch 1/20
100000/100000 [==============================] - 9s 93us/step - loss: 0.2397 - acc: 0.7424 - val_loss: 0.0929 - val_acc: 0.9356
Epoch 2/20
100000/100000 [==============================] - 5s 52us/step - loss: 0.0732 - acc: 0.9445 - val_loss: 0.0608 - val_acc: 0.9499
Epoch 3/20
100000/100000 [==============================] - 5s 53us/step - loss: 0.0547 - acc: 0.9544 - val_loss: 0.0490 - val_acc: 0.9577
Epoch 4/20
100000/100000 [==============================] - 5s 52us/step - loss: 0.0454 - acc: 0.9609 - val_loss: 0.0406 - val_acc: 0.9642
Epoch 5/20
100000/100000 [==============================] - 5s 52us/step - loss: 0.0395 - acc: 0.9639 - val_loss: 0.0420 - val_acc: 0.9581
Epoch 6/20
100000/100000 [==============================] - 5s 52us/step - loss: 0.0348 - acc: 0.9728 - val_loss: 0.0323 - val_acc: 0.9821
Epoch 7/20
100000/100000 [==============================] - 5s 52us/step - loss: 0.0311 - acc: 0.9814 - val_loss: 0

In [0]:
### DATOS TEST


X_test = np.zeros( (n_test, numk, 2 ) ) ## Lista de Datos para entrenar, en el format que necesitamos

data_raww = np.loadtxt("data_G_Test.txt")       ## Lista de Datos sin formato

for i in range(n_test) :
    count = int(i*2)
    X_test[i]= np.column_stack( ( data_raw[:,count] , data_raw[:,count+1] ) )
    

y_test= np.loadtxt("list_wn_G.txt_test")

X_test = X_test.reshape(n_test, numk, 2, 1).astype('float32')

print (X_test.shape)
print (y_test)

(20000, 33, 2, 1)
[-1. -1. -1. ...  0.  0.  0.]


In [0]:
model.evaluate(X_test, y_test)

20000/20000 [==============================] - 1s 58us/step


[1.0824656362056733, 0.37935]